In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import soundfile as sf
from scipy.signal import butter, filtfilt, hilbert, spectrogram, freqz, sosfiltfilt
from scipy.ndimage import gaussian_filter1d
from IPython.display import Audio, display
from datetime import datetime, timedelta

#### Testing JPM digital signal processing

In [ ]:
# load FLAC file
input_flac_file = "ManualRockMovements-1/audio/Nicole_091823-000100.flac"
data, samplerate = sf.read(input_flac_file)

# if stereo (more than one channel), keep only one channel
if data.ndim > 1:
    data = data[:,0] # keep the left channel, which in this case is the impact pipe

Gain Compensation

In [ ]:
preamp_gain_db = 23  # preamp gain in dB for this recording
linear_gain = 10**(preamp_gain_db / 20)
calibrated_data = data / linear_gain

print(f"Sampling Rate: {samplerate} Hz")
print(f"Original data shape: {data.shape}")
print(f"Linear gain factor: {linear_gain:.2f}")

# plot the raw and calibrated waveforms for verification
time = np.arange(len(data)) / samplerate
# note this took 2-minutes to run
plt.figure(figsize=(12, 6))
plt.plot(time, data, label="Raw Digital Data")
plt.plot(time, calibrated_data, label="Gain-Compensated Data")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Original vs. Gain-Compensated Waveform")
plt.legend(loc='upper left')
plt.show()

Spectrogram (for frequency high and low cut selection)

In [ ]:
frequencies, times, Sxx = spectrogram(calibrated_data, samplerate, nperseg=2048)
plt.figure(figsize=(15, 6))
plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
plt.colorbar(label='Intensity [dB]') 
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Acoustic Signal Spectrogram')
plt.ylim([0, samplerate / 2])  # limit y-axis to Nyquist frequency
plt.show()

Time averaged frequency spectrum

In [ ]:
frequencies, times, Sxx = spectrogram(calibrated_data, samplerate, nperseg=2048)

# average spectrum over time
cols = np.arange(len(times))
spec_avg = Sxx[:, cols].mean(axis=1)
spec_sm = gaussian_filter1d(spec_avg, 2) # smooth with gaussian filter; sigma=2

# plot the averaged and smoothed spectrum
plt.figure(figsize=(7,3.5))
plt.plot(frequencies, spec_sm, label="Smoothed avg spectrum")
plt.xlim(0, 2000)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Total Calibration Sample: Averaged Spectrum")
plt.tight_layout()
plt.show()

FFT frequency spectrum

In [ ]:
# FFT-based frequency spectrum
fft_data = np.fft.fft(calibrated_data)
frequencies_fft = np.fft.fftfreq(len(calibrated_data), 1/samplerate)

# take only positive frequencies and convert to magnitude
positive_freq_mask = frequencies_fft >= 0
frequencies_fft = frequencies_fft[positive_freq_mask]
magnitude_spectrum = np.abs(fft_data[positive_freq_mask])
spec_sm_fft = gaussian_filter1d(magnitude_spectrum, 500)  # smooth with gaussian filter; sigma=500

# plot FFT-based averaged and smoothed spectrum
plt.figure(figsize=(7,3.5))
plt.plot(frequencies_fft, spec_sm_fft, label="Smoothed FFT spectrum")
plt.xlim(0, 2000)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Total Calibration Sample: FFT-Based Spectrum")
plt.tight_layout()
plt.show()

Functions to select frequency band 

In [ ]:
def band_sos(low, high, fs, order=4):
    nyq = 0.5 * fs
    lown = max(low / nyq, 1e-6) # avoid 0 or negative
    highn = min(high / nyq, 0.999999) # avoid >1
    return butter(order, [lown := lown, highn], btype='band', output='sos') # bandpass filter

def apply_band_and_energy(x, fs, band, order=4):
    low, high = band 
    sos = band_sos(low, high, fs, order=order)
    y = sosfiltfilt(sos, x)
    # energy (RMS^2) overall and in sliding windows if desired
    energy = np.mean(y**2)
    return y, energy

def band_test_and_plot(x, fs, bands, event_window=None, quiet_window=None, nperseg=2048, noverlap=None):
    if noverlap is None:
        noverlap = nperseg // 2
    results = {}
    for band in bands:
        y, energy = apply_band_and_energy(x, fs, band)
        # compute spectrogram for plotting
        f, t, Sxx = spectrogram(y, fs=fs, window='hann', nperseg=nperseg, noverlap=noverlap, scaling='density', mode='magnitude')
        results[band] = {'y': y, 'energy': energy, 'f': f, 't': t, 'Sxx': Sxx}

    # compute event and quiet band energies if windows provided
    if event_window is not None:
        t0, t1 = event_window
        idx0 = int(t0 * fs); idx1 = int(t1 * fs)
    else:
        idx0, idx1 = 0, len(x)
    if quiet_window is not None:
        q0, q1 = quiet_window
        qidx0, qidx1 = int(q0*fs), int(q1*fs)
    else:
        # pick a quiet stretch before the event if possible
        qidx0, qidx1 = 0, min(int(0.5*fs), len(x))

    print("Band energy summary (mean squared) and event/quiet ratios:")
    eps = np.finfo(float).eps # small number to avoid div by 0
    for band, info in results.items():
        y = info['y'] # band-passed signal
        event_energy = np.mean(y[idx0:idx1]**2) # average power during event
        quiet_energy = np.median(y[qidx0:qidx1]**2)     # robust noise estimate
        quiet_energy = max(quiet_energy, eps)           # avoid division by 0
        ratio = event_energy / quiet_energy
        snr_db = 10.0 * np.log10(ratio)
        print(f"{band[0]:4.0f}-{band[1]:4.0f} Hz:"
            f" total {info['energy']:.3e},"
            f" event_power {event_energy:.3e},"
            f" quiet_power {quiet_energy:.3e},"
            f" ratio {ratio:.2f}, snr={snr_db:.1f} dB")

    # plot spectrograms side-by-side for visual comparison
    n = len(bands)
    plt.figure(figsize=(12, 3*n))
    for i, band in enumerate(bands, start=1):
        info = results[band]
        plt.subplot(n, 1, i)
        plt.pcolormesh(info['t'], info['f'], 10*np.log10(info['Sxx'] + 1e-12), shading='auto')
        plt.ylim(0, min(3000, fs/2))
        plt.title(f"Band {band[0]}-{band[1]} Hz spectrogram")
        plt.ylabel("Freq (Hz)")
        plt.colorbar(label='dB')
    plt.xlabel("Time (s)")
    plt.tight_layout()
    plt.show()

    return results

In [ ]:
bands = [(20,200), (700,1500), (20,2000)]
results = band_test_and_plot(calibrated_data, samplerate, bands, event_window=(11,850), quiet_window=(1100,1200))

Play filtered recordings for verification

In [ ]:
# normalize to avoid clipping
def play_array(x, fs, norm=True):
    xr = x / (np.max(np.abs(x)) + 1e-12) if norm else x
    display(Audio(xr, rate=fs))

# play original
play_array(calibrated_data, samplerate)

In [ ]:
# low band (20-200 Hz)
low = apply_band_and_energy(calibrated_data, samplerate, (20,200))[0] 
play_array(low, samplerate)

In [ ]:
# mid band (700-1500 Hz)
mid = apply_band_and_energy(calibrated_data, samplerate, (700,1500))[0]
play_array(mid, samplerate)

### Band-Pass Filtering 

Need to figure out the order (3, 4 or 5), so am going to test and filter using all three

In [ ]:
def band_pass_filter(data, samplerate, low, high, order):
    nyquist = 0.5 * samplerate
    low = low / nyquist
    high = high / nyquist
    b, a = butter(order, [low, high], btype='band') # get filter coefficients
    filtered_data = filtfilt(b, a, data)
    return filtered_data

In [ ]:
# plotting the filters
lowcut = 700        # low cutoff frequency (Hz)
highcut = 1200       # high cutoff frequency (Hz)
orders = [3, 4, 5, 6]  # filter orders to compare
nyquist = 0.5 * samplerate
plt.figure(figsize=(7, 4))

for order in orders:
    # get the filter coefficients
    b, a = butter(order, [lowcut / nyquist, highcut / nyquist], btype='band')
    # calculate the frequency response
    w, h = freqz(b, a, worN=2000) # worN determines the number of points at which to evaluate the response

    # convert angular frequency to Hz and magnitude to a linear scale (0-1)
    frequencies = w * samplerate / (2 * np.pi)
    magnitude_response = np.abs(h)

    # plot the magnitude response
    plt.plot(frequencies, magnitude_response, label=f'Order {order}')

# add vertical line at cutoff frequencies
plt.axvline(lowcut, color='gray', linestyle='--', label='Low Cutoff')
plt.axvline(highcut, color='gray', linestyle='--', label='High Cutoff')
plt.title('Butterworth Band-Pass Filter Frequency Response')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude')
plt.grid(True)
plt.legend()
plt.xlim(0, 3000)  # Adjust x-axis limits to better visualize the passband
plt.ylim(0, 1.1)   # Adjust y-axis limits
plt.show()

In [ ]:
lowcut = 700  # low cutoff frequency in Hz
highcut = 1200 # high cutoff frequency in Hz

data_3 = band_pass_filter(calibrated_data, samplerate, lowcut, highcut, order=3)
data_4 = band_pass_filter(calibrated_data, samplerate, lowcut, highcut, order=4)
data_5 = band_pass_filter(calibrated_data, samplerate, lowcut, highcut, order=5)



In [ ]:
# Visualize the filtered signal
plt.figure(figsize=(12, 6))
plt.plot(time, calibrated_data, label="Gain-Compensated Data", color='orange', alpha=0.5)
plt.plot(time, filtered_data, label="Band-Pass Filtered", color='blue', alpha=0.5)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Band-Pass Filtered Signal")
plt.legend()
plt.show()

### Envelope Detection

In [ ]:
# using the hilbert transform 
analytic_signal = hilbert(filtered_data)
envelope_hilbert = np.abs(analytic_signal)

# using rectification and low-pass filter
cutoff_freq_envelope = 100 # # very low low-pass filter to smooth the rectified signal
low_pass_cutoff = cutoff_freq_envelope / nyquist
b_lp, a_lp = butter(2, low_pass_cutoff, btype='low')
envelope_rectified = filtfilt(b_lp, a_lp, np.abs(filtered_data))

# visualize the envelopes
plt.figure(figsize=(12, 6))
plt.plot(time, filtered_data, 'b-', label='Filtered Signal', alpha=0.5)
plt.plot(time, envelope_hilbert, 'g-', linewidth=2, label='Hilbert Envelope')
plt.plot(time, envelope_rectified, 'r-', linewidth=2, label='Rectified Envelope')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Signal Envelope")
plt.legend()
plt.show()

### Pulse counting and Rectangular waveform

In [ ]:
from scipy.signal import find_peaks 

# Step 4: Pulse Counting
# The threshold should be calibrated based on background noise analysis.
peak_threshold = 0.0002
# The distance parameter is crucial for avoiding double counts on a single pulse.
# Calculate a sensible distance based on expected pulse duration (e.g., 5 ms).
min_pulse_duration = 0.0001 # seconds
min_distance_samples = int(min_pulse_duration * samplerate)

# Find peaks in the envelope
peaks, properties = find_peaks(envelope_hilbert, height=peak_threshold, distance=min_distance_samples)

print(f"Detected {len(peaks)} pulses.")

# Create the final rectangular waveform
pulse_width_samples = 500 # NOTE: pulse width should be defined based on the signal characteristic, need to change!! 
rectangular_waveform = np.zeros(len(filtered_data)) 
for peak_index in peaks:
    start_idx = max(0, peak_index - pulse_width_samples // 2)
    end_idx = min(len(filtered_data), peak_index + pulse_width_samples // 2)
    rectangular_waveform[start_idx:end_idx] = 0.0002

# Visualize the final rectangular waveform
plt.figure(figsize=(12, 6))
plt.plot(time, rectangular_waveform, 'r-', label="Final Rectangular Waveform")
plt.plot(time, envelope_hilbert, 'b--', alpha=0.4, label='Hilbert Envelope')
plt.plot(time, envelope_rectified, 'g-', alpha=0.99, label='Rectified Envelope')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Acoustic Rectangular Waveform from Pulses")
plt.legend()
plt.show() 


In [ ]:
# Zoom in on 1-2 seconds
start_time = 1.35  # seconds
end_time = 2    # seconds
# Convert to integer indices
start_idx = int(start_time * samplerate)
end_idx = int(end_time * samplerate)
# Slice the data for the zoomed view
time_zoom = time[start_idx:end_idx]
rectangular_waveform_zoom = rectangular_waveform[start_idx:end_idx]
envelope_hilbert_zoom = envelope_hilbert[start_idx:end_idx]
# Visualize the zoomed rectangular waveform
plt.figure(figsize=(12, 6))
plt.plot(time_zoom, rectangular_waveform_zoom, 'r-', label="Final Rectangular Waveform", linewidth=2)
plt.plot(time_zoom, envelope_hilbert_zoom, 'b--', alpha=0.5, label='Signal Envelope')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (1 = Pulse, 0 = Silence)")
plt.title("Acoustic Rectangular Waveform from Pulses (1-2 seconds)")
plt.legend()
plt.grid(True)
plt.show()

# Testing Manual Particle Movement Experiment Calibration


## MRM 1 - Japanese Pipe Micriphone

In [ ]:
filepath = 'ManualRockMovements-1/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()


The left channel is the impact pipe. This is true for all calibration recordings for H1 in 2023

#### Compute and plotting envelope

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

Plotting envelope and packets

In [ ]:
time = np.arange(len(y)) / sr

# plot
plt.figure(figsize=(15, 5))
plt.plot(time, y, color='black', linewidth=0.6, label="raw signal")
plt.plot(time, envelope, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# shade packet zones
for s, e in zip(packet_starts, packet_ends):
    plt.axvspan(time[s], time[e], color='lightcoral', alpha=0.4)

# De-duplicate Δtₚ label in legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]  # safety check
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Full Signal Packet Detection — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
start_time = 0  # in seconds
end_time = 1100  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Zoomed Packet Detection — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_amplitude_zoom = np.max(np.abs(envelope_zoom))
print(f"Max amplitude in the zoomed-in window: {max_amplitude_zoom:.2f}")

plt.show()


In [ ]:
# how many packets were detected?
print(f"Detected {len(packet_starts)} packets in {len(y)/sr:.2f} seconds of audio.")

Plotting and Saving Rock Movement Calibrations

In [ ]:
# import csv
csv_path = 'ManualRockMovements-1/mov1_pipe_particles.csv'
times_df = pd.read_csv(csv_path)

# Make sure "plots" folder exists
os.makedirs('ManualRockMovements-1/plots', exist_ok=True)

# Iterate through each row of the dataframe
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']
    
    # Convert times to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # Slice the arrays
    y_zoom = y[start_idx:end_idx]
    envelope_zoom = envelope[start_idx:end_idx]
    time_zoom = np.arange(start_idx, end_idx) / sr

    # Create the plot
    plt.figure(figsize=(15, 5))
    plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
    plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
    plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

    # Shade packets only if within the zoomed window
    for start, end in zip(packet_starts, packet_ends):
        if start >= start_idx and end <= end_idx:
            plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

    # Clean up legend
    handles, labels = plt.gca().get_legend_handles_labels()
    labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.xlabel("time (s)")
    plt.ylabel("amplitude")
    plt.title(f"Zoomed Packet Detection — Particle ID: {id_num} — {start_time}s to {end_time}s — Channel: {use_channel}")
    plt.grid(True)
    plt.tight_layout()

    # Save the figure
    plot_filename = f"ManualRockMovements-1/plots/pipe_signal_{id_num}.png"
    plt.savefig(plot_filename, dpi=300)
    plt.close()

    print(f"Saved plot for group {id_num} to {plot_filename}")

Displaying the max amplitude for each particle ID

In [ ]:
# import csv
csv_path = 'ManualRockMovements-1/mov1_pipe_particles.csv'
times_df = pd.read_csv(csv_path)

# loop through each row of the DataFrame
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the raw signal
    y_zoom = y[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(y_zoom))

    print(f"Particle ID {id_num}: Max Raw amplitude = {max_amp:.4f}")


In [ ]:
# loop through each row of the DataFrame
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the envelope
    envelope_zoom = envelope[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(envelope_zoom))

    print(f"Particle ID {id_num}: Max Envelope amplitude = {max_amp:.4f}")

## MRM 1 - Hydrophone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the hydrophone

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

Plotting envelope and packets

In [ ]:
time = np.arange(len(y)) / sr

# plot
plt.figure(figsize=(15, 5))
plt.plot(time, y, color='black', linewidth=0.6, label="raw signal")
plt.plot(time, envelope, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# shade packet zones
for s, e in zip(packet_starts, packet_ends):
    plt.axvspan(time[s], time[e], color='lightcoral', alpha=0.4)

# De-duplicate Δtₚ label in legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]  # safety check
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Full Signal Packet Detection — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()
plt.show()

Plotting and Saving Rock Movement Calibrations

In [ ]:
# import csv
csv_path = 'ManualRockMovements-1/mov1_mic_particles.csv'
times_df = pd.read_csv(csv_path)

# Make sure "plots" folder exists
os.makedirs('ManualRockMovements-1/plots/hydrophones', exist_ok=True)

# Iterate through each row of the dataframe
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']
    
    # Convert times to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # Slice the arrays
    y_zoom = y[start_idx:end_idx]
    envelope_zoom = envelope[start_idx:end_idx]
    time_zoom = np.arange(start_idx, end_idx) / sr

    # Create the plot
    plt.figure(figsize=(15, 5))
    plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
    plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
    plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

    # Shade packets only if within the zoomed window
    for start, end in zip(packet_starts, packet_ends):
        if start >= start_idx and end <= end_idx:
            plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

    # Clean up legend
    handles, labels = plt.gca().get_legend_handles_labels()
    labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.xlabel("time (s)")
    plt.ylabel("amplitude")
    plt.title(f"Zoomed Packet Detection — Particle ID: {id_num} — {start_time}s to {end_time}s — Channel: {use_channel}")
    plt.grid(True)
    plt.tight_layout()

    # Save the figure
    plot_filename = f"ManualRockMovements-1/plots/hydrophones/mic_signal_{id_num}.png"
    plt.savefig(plot_filename, dpi=300)
    plt.close()

Displaying the max amplitude for each particle ID 

In [ ]:
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the raw signal
    y_zoom = y[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(y_zoom))

    print(f"Particle ID {id_num}: Max Raw amplitude = {max_amp:.4f}")  

In [ ]:
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the envelope signal
    envelope_zoom = envelope[start_idx:end_idx]

    # get max amplitude from the envelope
    max_amp = np.max(np.abs(envelope_zoom))

    print(f"Particle ID {id_num}: Max Envelope amplitude = {max_amp:.4f}")


## MRM 2 - Impact Pipe

In [ ]:
filepath = 'ManualRockMovements-2/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()


Left channel is impact pipe also, since these experiments were on the same day MRM1

#### Compute and plotting envelope

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

Plotting envelope and packets

In [ ]:
start_time = 0  # in seconds
end_time = 400  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Zoomed Packet Detection — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_amplitude_zoom = np.max(np.abs(envelope_zoom))
print(f"Max amplitude in the zoomed-in window: {max_amplitude_zoom:.2f}")

plt.show()


In [ ]:
# how many packets were detected?
print(f"Detected {len(packet_starts)} packets in {len(y)/sr:.2f} seconds of audio.")

Plotting and saving group impacts

In [ ]:
# import csv
csv_path = 'ManualRockMovements-2/mov2_pipe_particles.csv'
times_df = pd.read_csv(csv_path)

# Make sure "plots" folder exists
os.makedirs('ManualRockMovements-2/plots', exist_ok=True)

# Iterate through each row of the dataframe
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']
    
    # Convert times to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # Slice the arrays
    y_zoom = y[start_idx:end_idx]
    envelope_zoom = envelope[start_idx:end_idx]
    time_zoom = np.arange(start_idx, end_idx) / sr

    # Create the plot
    plt.figure(figsize=(15, 5))
    plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
    plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
    plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

    # Shade packets only if within the zoomed window
    for start, end in zip(packet_starts, packet_ends):
        if start >= start_idx and end <= end_idx:
            plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

    # Clean up legend
    handles, labels = plt.gca().get_legend_handles_labels()
    labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.xlabel("time (s)")
    plt.ylabel("amplitude")
    plt.title(f"Zoomed Packet Detection — Group ID: {id_num} — {start_time}s to {end_time}s — Channel: {use_channel}")
    plt.grid(True)
    plt.tight_layout()

    # Save the figure
    plot_filename = f"ManualRockMovements-2/plots/pipe_groupsignal_{id_num}.png"
    plt.savefig(plot_filename, dpi=300)
    plt.close()

    print(f"Saved plot for group {id_num} to {plot_filename}")

Displaying the max amplitude for each particle ID

In [ ]:
# import csv
csv_path = 'ManualRockMovements-2/mov2_pipe_particles.csv'
times_df = pd.read_csv(csv_path)

# loop through each row of the DataFrame
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the raw signal
    y_zoom = y[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(y_zoom))

    print(f"Group ID {id_num}: Max Raw amplitude = {max_amp:.4f}")


In [ ]:
# loop through each row of the DataFrame
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the envelope
    envelope_zoom = envelope[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(envelope_zoom))

    print(f"Group ID {id_num}: Max Envelope amplitude = {max_amp:.4f}")

## MRM 2 - Hydrophone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the hydrophone

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

Plotting envelope and packets

In [ ]:
start_time = 0  # in seconds
end_time = 400  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Zoomed Packet Detection — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_amplitude_zoom = np.max(np.abs(envelope_zoom))
print(f"Max amplitude in the zoomed-in window: {max_amplitude_zoom:.2f}")

plt.show()

Plotting and saving groups

In [ ]:
# import csv
csv_path = 'ManualRockMovements-2/mov2_mic_particles.csv'
times_df = pd.read_csv(csv_path)

# Make sure "plots" folder exists
os.makedirs('ManualRockMovements-2/plots/hydrophones', exist_ok=True)

# Iterate through each row of the dataframe
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']
    
    # Convert times to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # Slice the arrays
    y_zoom = y[start_idx:end_idx]
    envelope_zoom = envelope[start_idx:end_idx]
    time_zoom = np.arange(start_idx, end_idx) / sr

    # Create the plot
    plt.figure(figsize=(15, 5))
    plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
    plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
    plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

    # Shade packets only if within the zoomed window
    for start, end in zip(packet_starts, packet_ends):
        if start >= start_idx and end <= end_idx:
            plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

    # Clean up legend
    handles, labels = plt.gca().get_legend_handles_labels()
    labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.xlabel("time (s)")
    plt.ylabel("amplitude")
    plt.title(f"Zoomed Packet Detection — Group ID: {id_num} — {start_time}s to {end_time}s — Channel: {use_channel}")
    plt.grid(True)
    plt.tight_layout()

    # Save the figure
    plot_filename = f"ManualRockMovements-2/plots/hydrophones/mic_groupsignal_{id_num}.png"
    plt.savefig(plot_filename, dpi=300)
    plt.close()

Displaying the max amplitude for each particle ID

In [ ]:
csv_path = 'ManualRockMovements-2/mov2_mic_particles.csv'
times_df = pd.read_csv(csv_path)

for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the raw signal
    y_zoom = y[start_idx:end_idx]

    # get max amplitude
    max_amp = np.max(np.abs(y_zoom))

    print(f"Group ID {id_num}: Max Raw amplitude = {max_amp:.4f}")  

In [ ]:
for idx, row in times_df.iterrows():
    id_num = row['id']
    start_time = row['start_time']
    end_time = row['end_time']

    # convert to sample indices
    start_idx = int(start_time * sr)
    end_idx = int(end_time * sr)

    # slice the envelope signal
    envelope_zoom = envelope[start_idx:end_idx]

    # get max amplitude from the envelope
    max_amp = np.max(np.abs(envelope_zoom))

    print(f"Particle ID {id_num}: Max Envelope amplitude = {max_amp:.4f}")


# Flood Experiments

### Flood 1

In [ ]:
filepath = 'EXP-09-20-23-FLOW1/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe recording: unusable
- For some reason there was a lot of static in the recordings, don't know if it was like a hardware thing? 
- Still gonna try to see if it gives me anything

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 180  # in seconds
end_time = 660  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 2 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()  

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 180  # in seconds
end_time = 660  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 2 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()  

### Flood 2

In [ ]:
filepath = 'EXP-09-20-23-FLOW2/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 200  # in seconds
end_time = 250  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 2 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()


microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 200  # in seconds
end_time = 250  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 2 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()


### Flood 3

In [ ]:
filepath = 'EXP-09-20-23-FLOW3/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 250  # in seconds
end_time = 350  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 3 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 250  # in seconds
end_time = 350  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 3 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

### Flood 4

In [ ]:
filepath = 'EXP-09-21-23-FLOW4/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe: no impact 
- Still going to try to get a max amplitude

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 600  # in seconds
end_time = 800  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 4 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 600  # in seconds
end_time = 800  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 4 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

### Flood 5

In [ ]:
filepath = 'EXP-09-21-23-FLOW5/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 50  # in seconds
end_time = 200  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 5 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 23   # left channel gain (dB)
right_gain_db = 4 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 50  # in seconds
end_time = 200  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 5 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

### Flood 6

In [ ]:
filepath = 'EXP-09-22-23-FLOW6/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 200  # in seconds
end_time = 400  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 6 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 200  # in seconds
end_time = 400  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 6 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

### Flood 7
These is no recording for this flood :(

### Flood 8

In [ ]:
filepath = 'EXP-09-22-23-FLOW8/audio/Nicole_091823-000100.flac'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 100  # in seconds
end_time = 300  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 8 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

mic

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 100  # in seconds
end_time = 300  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 8 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

### Flood 9

In [ ]:
filepath = 'EXP-09-22-23-FLOW9/Nicole_414.wav'

# load stereo signal
signal, sr = librosa.load(filepath, sr=None, mono=False)
left_channel = signal[0]
right_channel = signal[1]

#create the time axis
duration = signal.shape[-1] / sr
time = np.linspace(0, duration, signal.shape[-1])

# plot the signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# plot the right channel (top)
axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
axes[0].set_title("Right Channel")
axes[0].set_ylabel("Amplitude")
axes[0].legend()

# plot the left channel (bottom)
axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
axes[1].set_title("Left Channel")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].legend()

# adjust layout to prevent overlap
plt.tight_layout()
plt.show()

pipe

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "left"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 50  # in seconds
end_time = 250  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 9 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()

microphone

In [ ]:
# === USER INPUT ===
left_gain_db = 30   # left channel gain (dB)
right_gain_db = 2 # right channel gain (dB)
use_channel = "right"  # which one was the impact pipe

# convert dB gain to linear - linear amplitude ratio
left_gain = 10 ** (left_gain_db / 20)
right_gain = 10 ** (right_gain_db / 20)

# choose channel and correct for gain
y = signal[0] if use_channel == "left" else signal[1]
gain_correction = left_gain if use_channel == "left" else right_gain
y = y / gain_correction  # undo preamp gain

# compute envelope
analytic_signal = scipy.signal.hilbert(y)
envelope = np.abs(analytic_signal)

# estimate dynamic threshold (adjust multiplier as needed)
noise_level = np.median(envelope)
threshold = 40 * noise_level

# detect packet start/end points
above_thresh = envelope > threshold # boolean array indicating where the envelope is above the threshold
packet_starts = np.where(np.diff(above_thresh.astype(int)) == 1)[0] # where the envelope crosses above the threshold
packet_ends = np.where(np.diff(above_thresh.astype(int)) == -1)[0] # where the envelope crosses below the threshold

# remove any packets that are too short (less than 0.1 seconds)
if packet_ends[0] < packet_starts[0]: 
    packet_ends = packet_ends[1:] 
if len(packet_starts) > len(packet_ends):
    packet_starts = packet_starts[:-1]

# extract max amplitudes per packet
max_amplitudes = [np.max(np.abs(y[start:end])) for start, end in zip(packet_starts, packet_ends)]

In [ ]:
start_time = 50  # in seconds
end_time = 250  # in seconds

# convert times to sample indices
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

# slice all relevant arrays to just this window
y_zoom = y[start_idx:end_idx]
envelope_zoom = envelope[start_idx:end_idx]
time_zoom = np.arange(start_idx, end_idx) / sr

# Plot (zoomed-in view) 
plt.figure(figsize=(15, 5))
plt.plot(time_zoom, y_zoom, color='black', linewidth=0.6, label="raw signal")
plt.plot(time_zoom, envelope_zoom, color='darkred', linewidth=1.0, label="envelope")
plt.axhline(threshold, color='black', linestyle='--', linewidth=1, label="threshold")

# only show shaded packets that fall within the zoomed-in range
for start, end in zip(packet_starts, packet_ends):
    if start >= start_idx and end <= end_idx:
        plt.axvspan(start / sr, end / sr, color='lightcoral', alpha=0.4)

# clean up legend
handles, labels = plt.gca().get_legend_handles_labels()
labels = ["Δtₚ" if "Patch" in str(h) else l for h, l in zip(handles, labels)]
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("time (s)")
plt.ylabel("amplitude")
plt.title(f"Flood 9 — {start_time}s to {end_time}s — Channel: {use_channel}")
plt.grid(True)
plt.tight_layout()

# calculate the max amplitude of envelope in this window
max_envelope_zoom = np.max(np.abs(envelope_zoom))
print(f"Max envelope amplitude in the zoomed-in window: {max_envelope_zoom:.4f}")
max_signal_zoom = np.max(np.abs(y_zoom))
print(f"Max raw signal amplitude in the zoomed-in window: {max_signal_zoom:.4f}")

plt.show()